In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, RocCurveDisplay
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import dump
# Loading the data  data-----------------------------------
df = pd.read_csv("flood_model_lagged.csv", parse_dates=["date"])

X = df.drop(columns=["date", "flood"])
y = df["flood"]

# ----------------------train test split -----------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


# ---------- loading the model (Gradient Boosting )-----------

gb = GradientBoostingClassifier(random_state=42)
gb.fit(X_train, y_train)
gb_preds = gb.predict(X_test)

print(" Gradient Boosting:")
dump(gb, "gb_flood_model.pkl")
print(classification_report(y_test, gb_preds))

# ---------- loading the model (XGBoost )-----------
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb.fit(X_train, y_train)
xgb_preds = xgb.predict(X_test)

print("\n XGBoost:")
dump(xgb, "xgb_flood_model.pkl")

print(classification_report(y_test, xgb_preds))


# ---------the part of  Plot ROC Curve------------

fig, ax = plt.subplots(figsize=(7, 5))
RocCurveDisplay.from_estimator(gb, X_test, y_test, ax=ax, name="Gradient Boosting")
RocCurveDisplay.from_estimator(xgb, X_test, y_test, ax=ax, name="XGBoost")
plt.title("ROC Curve Comparison")
plt.show()


# Cross-validation part --------------------

gb_cv = cross_val_score(gb, X, y, scoring='f1', cv=5)
xgb_cv = cross_val_score(xgb, X, y, scoring='f1', cv=5)

print(" Cross-Validation F1 Score:")
print(f"Gradient Boosting: Mean F1 = {gb_cv.mean():.4f}")
print(f"XGBoost:          Mean F1 = {xgb_cv.mean():.4f}")


FileNotFoundError: [Errno 2] No such file or directory: 'flood_model_lagged.csv'